In [ ]:
import pandas as pd
reviews_dataset = pd.DataFrame(columns=['id', 'origin','date','review_text','clean_text','rating','location_lat','location_long','venue_category','type','topic','sentiment'])


In [ ]:
#AIRBNB DATASET

import pandas as pd
from utils import language_det,clean_reviews
airbnb_dataset = pd.DataFrame(columns=['id', 'origin','date','review_text','clean_text','rating','location_lat','location_long','venue_category','type','topic','sentiment'])
dfReviews = pd.DataFrame(columns=['id', 'origin','review_text', 'venue_category'])
dfListings = pd.read_csv('airbnb/listings.csv', usecols = ['id','latitude','longitude','property_type'])
dfReviews = pd.read_csv('airbnb/en_reviews.csv',usecols = ['id','listing_id','comments','date'])
#dfReviews = dfReviews[dfReviews['comments'].notnull()] #keep only non Nan rows

#display(dfReviews['comments'].str.len())
#dfReviews[dfReviews['comments'] == ]
#print(detect(dfReviews['comments'][0]))
#dfReviews = dfReviews[dfReviews['comments'].apply(language_det) != 'en']

dfReviews = dfReviews.rename(columns={"id": "review_id", "listing_id": "id"})
dfinal = pd.merge(dfListings, dfReviews, on='id')
dfinal['origin'] = 'airbnb'
dfinal['type'] = 'accommodation'

#dfinal.columns = ['listing_id', 'location_lat',"location_lng", "venue_category","id","date","review_text","origin","type" ]
#dfinal = dfinal.drop('listing_id', 1)
airbnb_dataset[['id','review_text','location_lat','location_long','origin','venue_category','date','type']] = dfinal[['review_id', 
                                                                    'comments','latitude','longitude',
                                                                    'origin','property_type','date','type']]
airbnb_dataset = airbnb_dataset[airbnb_dataset['review_text'].notnull()] #keep only non Nan rows
airbnb_dataset = clean_reviews(airbnb_dataset)



In [ ]:
#AIRBNB SENTIMENT ANALYSIS
from utils import nltk_vader

airbnb_list = airbnb_dataset['clean_text'].tolist()
final_airbnb_corpus  = [" ".join(str(word) for word in review) for review in airbnb_list]
airbnb_dataset['sentiment'] = nltk_vader(final_airbnb_corpus)
display(airbnb_dataset.head())


In [ ]:
#TWITTER DATASET
from utils import clean_reviews
from ast import literal_eval
import os, json
import pandas as pd
import time
import datetime

twitter_dataset = pd.DataFrame(columns=['id', 'origin','date','review_text','clean_text','rating','location_lat','location_long','venue_category','type','topic','sentiment'])

# this finds our json files
path_to_json_tweets = 'tweets/'
json_tweets = [json_tweet for json_tweet in os.listdir(path_to_json_tweets)]
tweets_pd = pd.DataFrame(columns=['id','text'])

#Find tweet's location
location_dict = {}
with open('lexicon/twitter_location_dict') as lexicon:
    for line in lexicon:
        key, value = line.strip().split(":")
        location_dict[key] = value

reviews_list = []
for index, js in enumerate(json_tweets):
    with open(os.path.join(path_to_json_tweets, js)) as json_file:
        tweet_df = json.load(json_file)
        location = ["37.973663","23.7325461"]
        for word in tweet_df['text'].split():
                if word in location_dict:
                    location = literal_eval(location_dict[word])
        tweet_date = time.strftime('%Y-%m-%d', time.strptime(tweet_df['created_at'],'%a %b %d %H:%M:%S +0000 %Y'))
        reviews_list.append({'id': tweet_df['id'], 'origin' : 'twitter','date': tweet_date,'review_text': tweet_df['text'],
                            'location_lat':location[0],'location_long':location[1]})

twitter_dataset = twitter_dataset.append(reviews_list,ignore_index=True)
twitter_dataset = clean_reviews(twitter_dataset)

In [ ]:
#TWEETS CLASSIFICATION
from utils import svm_classification,prepare_training_set, prepare_test_set, nltk_vader,dominant_sentiment


#training dataset
train_tweets_df = pd.read_csv('traindata/train2017.tsv', sep='\t', usecols = [2,3],names = ['sentiment', 'review_text'] ,header = None)
train_tweets_df = clean_reviews(train_tweets_df)
final_train_corpus = prepare_training_set(train_tweets_df)

#test dataset
final_test_corpus = prepare_test_set(twitter_dataset)


#labels
sent_map = {"positive":1, "neutral":0, "negative":-1}
y_train_tweets_labels = [sent_map[sentiment] for sentiment in train_tweets_df['sentiment'].tolist()] #sentiments

#nltk vader
nltk_vader_list = nltk_vader(final_train_corpus)

#SVM AGORITHM FOR CLASSIFICATION
bow_y_test_tweets_labels = svm_classification(final_train_corpus, final_test_corpus, y_train_tweets_labels,'BOW')
tfidf_y_test_tweets_labels =  svm_classification(final_train_corpus, final_test_corpus, y_train_tweets_labels,'TFIDF')
twitter_dataset['sentiment'] = dominant_sentiment(bow_y_test_tweets_labels,tfidf_y_test_tweets_labels,nltk_vader_list)
display(twitter_dataset.head())

In [ ]:
#FOURSQUARE DATASET

import os, json
import pandas as pd
from datetime import datetime
from utils import clean_reviews
foursquare_dataset = pd.DataFrame(columns=['id', 'origin','date','review_text','clean_text','rating','location_lat','location_long','venue_category','type','topic','sentiment'])
            
# this finds our json files
root = 'foursquare/'
reviews_list = []
for path, dirs, files in os.walk(root):
    for name in files:
        if name.endswith('.json'):
            with open(os.path.join(path, name)) as json_file:
                foursquare_df = json.load(json_file)
                for tip in foursquare_df['venue']['tips']['groups'][0]['items']:  #2 most upvoted tips
                    if('lang' in tip):
                        if(tip['lang']=='en'):  #keep english only
                            #print(tip['text'])
                            foursquare_date  = datetime.utcfromtimestamp(tip['createdAt']).strftime('%Y-%m-%d')
                            reviews_list.append({'id': foursquare_df['venue']['id'], 
                                      'origin' : 'foursquare',
                                      'date': foursquare_date,
                                      'review_text': tip['text'],
                                      'location_lat':foursquare_df['venue']['location']['lat'],
                                      'location_long':foursquare_df['venue']['location']['lng'],
                                      'type':os.path.basename(path),
                                      'venue_category':foursquare_df['venue']['categories'][0]['name']})
foursquare_dataset = foursquare_dataset.append(reviews_list,ignore_index=True)
foursquare_dataset = clean_reviews(foursquare_dataset)

In [ ]:
#FOURSQUARE CLASSIFICATION
from utils import svm_classification,prepare_training_set, prepare_test_set, nltk_vader,dominant_sentiment

#training dataset
train_foursquare_df = pd.read_csv('traindata/en_foursquare_train.csv', sep=',', usecols = [0,1],names = ['review_text', 'sentiment'],skiprows=1)
train_foursquare_df = clean_reviews(train_foursquare_df)
final_train_corpus = prepare_training_set(train_foursquare_df)

#test dataset
final_test_corpus = prepare_test_set(foursquare_dataset)


#labels
y_train_foursquare_labels = train_foursquare_df['sentiment'].tolist() #sentiments

#nltk vader
nltk_vader_list = nltk_vader(final_train_corpus)

bow_y_test_foursquare_labels = svm_classification(final_train_corpus, final_test_corpus, y_train_foursquare_labels,'BOW')
tfidf_y_test_foursquare_labels =  svm_classification(final_train_corpus, final_test_corpus, y_train_foursquare_labels,'TFIDF')

foursquare_dataset['sentiment'] = dominant_sentiment(bow_y_test_foursquare_labels,tfidf_y_test_foursquare_labels,nltk_vader_list)
display(foursquare_dataset.head())

In [ ]:
#MAPS DATASET

import os, json
import pandas as pd
import time
import datetime

maps_dataset = pd.DataFrame(columns=['id', 'origin','date','review_text','clean_text','rating','location_lat','location_long','venue_category','type','topic','sentiment'])

# this finds our json files
root = 'places/'
reviews_list = []
for path, dirs, files in os.walk(root):
    for name in files:
        if name.endswith('.json'):
            with open(os.path.join(path, name)) as json_file:
                maps_df = json.load(json_file)
                for review in maps_df['reviews']:  #5 most useful tips
                    if('language' in review):
                        if(review['language']=='en'):  #keep english only
                            if float(maps_df['rating'])<3 and float(maps_df['rating'])>0:
                                sentiment = '-1'
                            elif float(maps_df['rating'])<5 and float(maps_df['rating'])>4 :
                                sentiment = '1'
                            else:
                                sentiment = '0'
                            reviews_list.append({'id': maps_df['id'], 
                                      'origin' : 'google_maps',
                                      'review_text': review['text'],
                                      'rating':maps_df['rating'],
                                      'location_lat':maps_df['location']['lat'],
                                      'location_long':maps_df['location']['lng'],
                                      'type':os.path.basename(path),
                                      'sentiment':sentiment,
                                      'venue_category':maps_df['type']})
        else:
            #data was not valid JSON
            continue

maps_dataset = maps_dataset.append(reviews_list,ignore_index=True)
maps_dataset = clean_reviews(maps_dataset)



In [ ]:
#LDA
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from utils import format_topics_sentences,topic_keywords
import pandas as pd
import ast


#topic dictionary
topic = []
counter = 0
for review in reviews_dataset['clean_text'].tolist():
    id2word = corpora.Dictionary([review])
    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in [review]]
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           per_word_topics=True);
    keywords = topic_keywords(lda_model, corpus,[review])
    counter +=1
    if counter % 1000 == 0:
        print(counter)
    topic.append(keywords.tolist())
reviews_dataset['topic'] = topic
reviews_dataset.to_csv(r'reviews_dataset_test.csv',columns=['id', 'origin','date','review_text','clean_text','rating','location_lat','location_long','venue_category','type','topic','address','sentiment'] ,index=False) #save reviews to file for future use


In [ ]:
#CONCAT NEW DATA TO EXISTING CSV
reviews_dataset = [maps_dataset,twitter_dataset,airbnb_dataset,foursquare_dataset]
reviews_dataset = pd.concat(reviews_dataset,ignore_index=True)
display(reviews_dataset[reviews_dataset['origin']=='airbnb'])




In [ ]:
#CONVERT COORDS TO AREAS

#FIND GENERAL LOCATION USING A LIBRARY THAT EXPLOITS AN OFFLINE DATABASE
import reverse_geocoder as rg

reviews_dataset["address"] = ""
llat = reviews_dataset['location_lat'].tolist()
llng = reviews_dataset['location_long'].tolist()
coordinates = tuple(zip(llat, llng))  
results = rg.search(coordinates)
cities = []
for city in results:
    cities.append(city.get('name'))
reviews_dataset['address'] = cities

#FIND PARTICUAR USING A WEB API (REALY SLOW PROCESS)
#TO MONITOR THE WHOLE PROCESS I HAVE SPLIT DATAFRAME INTO CHUNKS
from geopy.geocoders import Nominatim
from functools import partial
from tqdm import tqdm
import time


tqdm.pandas()
geolocator = Nominatim(timeout = 3)

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)



import glob

all_files = glob.glob('df_chunks' + "/*.csv")
for filename in all_files:
    reviews_test = pd.read_csv(filename)
    reviews_test['address'] = reviews_test.apply(lambda row:[geocode((str(row['location_lat']) +","+ str(row['location_long'])),language= 'en')] 
                                                   if row['address'] == 'Athens' else row['address'], axis=1)
    reviews_test.to_csv(filename,columns=['id', 'origin','date','review_text','clean_text','rating','location_lat','location_long','venue_category','type','topic','address','sentiment'] ,index=False) #save reviews to file for future use


In [ ]:
#PROCESS THE RETURNED RESULTS FROM NOMINATIM
from utils import specify_location
reviews_dataset['address'] = reviews_dataset_coords['address'].apply(specify_location)


In [ ]:
import json

with open('geojson/synoikies.geojson') as f:
    data = json.load(f)

for feature in data['features']:
    print(feature['properties']['SINIKIA_EN'])






In [ ]:
#CONCAT NEW DATA TO EXISTING CSV
import glob
import pandas as pd

all_files = glob.glob('df_chunks' + "/*.csv")
df_list = []
for filename in all_files:
    df_list.append(pd.read_csv(filename))
    reviews_dataset_coords = pd.concat(df_list,ignore_index=True)
    reviews_dataset_coords.to_csv(r'reviews_dataset_coords.csv',columns=['id', 'origin','date','review_text','clean_text','rating','location_lat','location_long','venue_category','type','topic','address','sentiment'] ,index=False) #save reviews to file for future use

In [ ]:
#WRITE DATAFRAME TO FILE
reviews_dataset.to_csv(r'reviews_dataset.csv',columns=['id', 'origin','date','review_text','clean_text','rating','location_lat','location_long','venue_category','type','topic','sentiment'] ,index=False) #save reviews to file for future use
#reviews_dataset_coords.to_csv(r'reviews_dataset.csv',columns=['id', 'origin','date','review_text','clean_text','rating','location_lat','location_long','venue_category','type','topic','address','sentiment'] ,index=False) #save reviews to file for future use